<h1 align = 'center'>Guessing Games</h1>
<h3 align = 'center'>machine learning, one step at a time</h3>
<h3 align = 'center'>Step 13. Tic-Tac-Toe</h3>

***13. Tic-tac-toe***

Tic-tac-toe doesn't need an introduction. Let's just play.

Make a move by typing the number of the space that you want to claim.

In [ ]:
from tictactoe import *
#
# we'll let the human player go first
#
g = Game(HumanPlayer(), MinimaxPlayer())
g.play()

In [ ]:
from tictactoe import *
#
# now we'll let the computer player go first.. 
# the first move might take a little while.
#
g = Game(MinimaxPlayer(), HumanPlayer())
g.play()

OK, so with a little practice, the best we can hope for is a tie game, because we are playing against a __Minimax__ player.

Who is this __MinimaxPlayer__ who knows so much about tic-tac-toe?

_And if this course is called __Guessing Games__, why play tic-tac-toe at all? What does that have to do with guessing?_

(and why are you asking so many questions all at once?)

Let's start with __Minimax__:

<font face=Times size=3><bold><blockquote>
    The Minimax algorithm is the most well-known strategy of play of two-player, zero-sum games. The minimax theorem was proven by John von Neumann in 1928. Minimax is a strategy of always minimizing the maximum possible loss which can result from a choice that a player makes.<p><br>
    Roberts, _Strategies of Play_, Stanford University Computer Science (1998-1999)
</blockquote></bold></font>
Or, one step at a time:
- a __two-player, zero-sum-game__ is a game where a player wins only by causing another player to lose.
- other examples of two-player, zero-sum games would be __singles tennis__ or __the cold war__.
- __John von Neumann__ was the Einstein of math, game theory, and computer science.
- __Minimax__ says "if you can't win right now, prevent your opponent from winning."
- __Minimax__ tries every possible move, and every possible response, every time.
- to function, __Minimax__ needs to know the rules of the game (in order to try every move).

In practice, a half-finished pass through the __Minimax__ tic-tac-toe algorithm looks something like this:
<img src="minimax-animation.png">
(image generated at http://www.algomation.com)<p>
__Minimax__ is the opposite of _machine learning_. Remember that we defined _machine learning_ as _the art of accumulating knowledge by learning from mistakes_? __Minimax__ doesn't make any mistakes. Ever.

So how can machine learning ever hope to defeat the evil __Minimax__?

(or OK, at least play to draw?)

Let's start with a look at a tic-tac-toe environment, which can present itself just like the maze, using __reset()__, __sample()__, and __step()__. To keep things simple, the tic-tac-toe environment always allows the machine learning player to play __X__ and therefore move first.

Let's start by retrieving our initial state, using __reset()__:

In [15]:
from tictactoe import *

########################################
#                                      #
# The TicTacToe framework can present  #
# itself just like the maze... using   #
# reset(), sample(), and step(action)  #
#                                      #
########################################

g = Game(EmptyPlayer()) # specifying only an opponent sets up system for reinforcement learning
state = g.reset()
print('the problem has', g.state_space(), 'states; initial state =', state)

the problem has 19683 states; initial state = 9841


Wow. We are in state 9841 out of 19683 possible states.

The good news is: when taking a machine learning approach, we don't care which state is which, or which one comes first (provided that our computer has enough memory to record them all).

But for the curious: there are 9 squares, and each square is either __X__, __O__, or __empty__; the framework assigns discrete values: __X=2, empty=1, O=0__, the state is represented as a 9-digit number, in base 3:

In [19]:
# you don't need to know these details to solve the problem!
# it's here to avoid the occasional github post saying: how do the states work...?

def calculate_state(board):
    sum = 0
    for n in range(len(board)):
        sum += board[n] * 3**n
    return sum

print("all O's   =",calculate_state([0,0,0,0,0,0,0,0,0]))
print("all X's   =",calculate_state([2,2,2,2,2,2,2,2,2]))
print("all empty =",calculate_state([1,1,1,1,1,1,1,1,1]))

all O's   = 0
all X's   = 19682
all empty = 9841


All we _really need to know_ is that our problem has a finite number of states and actions, and we can rely on the framework to provide a sample action:

In [20]:
from tictactoe import *
g = Game(EmptyPlayer())
print('total states =', g.state_space(), 'total actions =', g.action_space())
for n in range(10):
    print('sample action:', g.sample())

total states = 19683 total actions = 9
sample action: 3
sample action: 4
sample action: 6
sample action: 1
sample action: 5
sample action: 7
sample action: 3
sample action: 0
sample action: 1
sample action: 6


Let's create __QPlayer__, a new type of player that, for starters, takes random actions.

We can implement a new player like this:

In [ ]:
class QPlayer(BasePlayer):
    
    